In [59]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

In [60]:
# Output File (CSV)
#output_data_file = "./output_data/cities.csv"

In [61]:
# Output File (CSV)
output_data_file = "/Users/vrmathur/Documents/Gtech/Python-api-Challenge/WeatherPy/output_data\my_weather_data.csv"
output_data_file_df = pd.read_csv(output_data_file)


output_data_file_df = output_data_file_df.dropna()
output_data_file_df.head()

,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Tuktoyaktuk,69.4541,-133.0374,24.94,84,45,7.36,CA,1619838000
1,1,Tarbagatay,51.4803,107.3602,48.85,48,47,1.23,RU,1619838000
2,2,Koygorodok,60.4450,50.9968,43.79,92,100,13.56,RU,1619838000
3,3,Port-Gentil,-0.7193,8.7815,81.10,86,47,6.73,GA,1619838000
4,4,Rikitea,-23.1203,-134.9692,79.18,79,93,19.98,PF,1619838000


## Humidity Heatmap

Configure gmaps.
Use the Lat and Lng as locations and Humidity as the weight.
Add Heatmap layer to map.

In [62]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [63]:
# Store 'Lat' and 'Lng' into  locations
#locations = output_data_file_df[["Lat", "Lng"]]
#Humidity = output_data_file_df["Humidity"].astype(float)

# Store 'Lat' and 'Lng' into  locations
locations = output_data_file_df[["Latitude", "Longitude"]]
humidity = output_data_file_df["Humidity"].astype(float)

In [64]:
# Plot Heatmap
fig = gmaps.figure(center = [0,0] ,zoom_level = 2)

#Create and add heat layer 
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                               dissipating=False, max_intensity=100,
                               point_radius = 4)
fig.add_layer(heat_layer)
#Display figure
fig

Figure(layout=FigureLayout(height='420px'))

## Create new DataFrame fitting weather criteria

Narrow down the cities to fit weather conditions.

Drop any rows will null values.


In [66]:
#NarrowDownData_df = output_data_file_df.loc[(output_data_file_df['Max Temp'] > 70) & (output_data_file_df['Max Temp'] < 80) & (output_data_file_df['Wind Speed'] < 10) & (output_data_file_df['Cloudiness'] == 0)]

In [71]:
#Drop any rows will null values
NarrowDownData_df = NarrowDownData_df.dropna()
NarrowDownData_df.head()

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel name
6,Morondava,-20.2833,44.2833,75.11,83,0,5.70,MG,1619838000,
71,Atuona,-9.8000,-139.0333,79.45,68,0,0.18,PF,1619838000,
128,Chaohu,31.6000,117.8667,72.55,51,0,1.99,CN,1619838000,
420,Buraidah,26.3260,43.9750,75.58,31,0,5.57,SA,1619838000,


## Hotel Map

Store into variable named hotel_df.

Add a "Hotel Name" column to the DataFrame.

Set parameters to search for hotels with 5000 meters.

Hit the Google Places API for each city's coordinates.

Store the first Hotel result into the DataFrame.

Plot markers on top of the heatmap.

In [75]:
hotel_df = NarrowDownData_df
hotel_df["Hotel name"] = ''
hotel_df.head()

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel name
6,Morondava,-20.2833,44.2833,75.11,83,0,5.70,MG,1619838000,
71,Atuona,-9.8000,-139.0333,79.45,68,0,0.18,PF,1619838000,
128,Chaohu,31.6000,117.8667,72.55,51,0,1.99,CN,1619838000,
420,Buraidah,26.3260,43.9750,75.58,31,0,5.57,SA,1619838000,


In [79]:
hotel_df = ideal_weather.reset_index(drop=True)
hotel_df["Hotel Name"] = ""

# params dictionary to update each iteration
params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}

# Use the lat/lng we recovered to identify hotels
for index, row in hotel_df.iterrows():
    
    # get lat, lng 
    lat = row["Latitude"]
    lng = row["Longitude"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotels" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # make request and print url
    hotel_name = requests.get(base_url, params=params)
    
    # convert to json
    hotel_name = hotel_name.json()
    print(json.dumps(hotel_name, indent=4, sort_keys=True))
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
hotel_df

{
    "error_message": "The provided API key is invalid.",
    "html_attributions": [],
    "results": [],
    "status": "REQUEST_DENIED"
}
Missing field/result... skipping.
{
    "error_message": "The provided API key is invalid.",
    "html_attributions": [],
    "results": [],
    "status": "REQUEST_DENIED"
}
Missing field/result... skipping.
{
    "error_message": "The provided API key is invalid.",
    "html_attributions": [],
    "results": [],
    "status": "REQUEST_DENIED"
}
Missing field/result... skipping.
{
    "error_message": "The provided API key is invalid.",
    "html_attributions": [],
    "results": [],
    "status": "REQUEST_DENIED"
}
Missing field/result... skipping.


,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,6,Morondava,-20.2833,44.2833,75.11,83,0,5.70,MG,1619838000,
1,71,Atuona,-9.8000,-139.0333,79.45,68,0,0.18,PF,1619838000,
2,128,Chaohu,31.6000,117.8667,72.55,51,0,1.99,CN,1619838000,
3,420,Buraidah,26.3260,43.9750,75.58,31,0,5.57,SA,1619838000,
